# Lista 7


In [159]:
from sympy import symbols, init_printing, dsolve
import sympy as sp
import pandas as pd

init_printing(use_unicode=True)

In [160]:
y = symbols("y", cls=sp.Function)
x = symbols("x", real=True)

## Kody funkcji


### Rozwiązanie dokładne **(RD)**


In [161]:
def fun_RD(f, x0, y0):
    return dsolve(sp.Eq(y(x).diff(x), f), y(x), ics={y(x0): y0})

### Rozwiązanie przybliżone analityczne **(RA)**


In [162]:
def fun_RA(f, x0, y0, n=5):
    pochodne_bez_podstawien = [f]
    for i in range(n + 1):
        pochodne_bez_podstawien.append(sp.diff(pochodne_bez_podstawien[-1], x))

    pochodne_od_x = [y0, f.subs({x: x0, y(x): y0})]
    for i in range(1, len(pochodne_bez_podstawien)):
        pochodna = pochodne_bez_podstawien[i]
        for j in range(i, 0, -1):
            pochodna = pochodna.subs(y(x).diff((x, j)), pochodne_od_x[j])
        pochodna = pochodna.subs(x, x0).subs(y(x0), y0)
        pochodne_od_x.append(pochodna.simplify())

    skladowe = [
        pochodne_od_x[ni] / sp.factorial(ni) * (x - x0) ** ni for ni in range(0, n + 1)
    ]

    return sum(skladowe)

### Rozwiązanie przybliżone numeryczne **(RN)**


In [163]:
def fun_RN(f, x0, y0, xn, h=0.2):
    # jeżeli wyliczanoo x jest większe lub równe szukanemu, to zwrcamy y
    # (w przypadku gdy kroki nie pokryją się z szukaną wartością,
    # zwrócona zostanie wartość dla najbliżeszego mniejszego kroku)
    if x0 >= xn:
        return y0
    y_k = y0 + h * f.subs({x: x0, y(x): y0})
    x_k = x0 + h
    return fun_RN(f, x_k, y_k, xn, h)

In [164]:
def fun_RN_iter(f, x0, y0, xn, h=0.2):
    while x0 < xn:
        y_k = y0 + h * f.subs({x: x0, y(x): y0})
        x0 += h
        y0 = y_k

    return y0

## Rozwiązania


### Podpunkt **(a)**


In [165]:
f = 2 * y(x) + sp.exp(x) - x
y0 = 1 / 4
x0 = 0

In [166]:
kolumny = [0, 0.2, 0.4, 0.6, 0.8, 1]
wiersze = ["RD", "RA", "RN"]
wartości = [
    [fun_RD(f, x0, y0).subs(x, x_val).evalf(5).rhs for x_val in kolumny],
    [fun_RA(f, x0, y0).subs(x, x_val).evalf(5) for x_val in kolumny],
    [round(fun_RN(f, x0, y0, x_val), 5) for x_val in kolumny],
]
table_data = [(wiersze[i], *wartości[i]) for i in range(len(wiersze))]
pd.DataFrame(table_data, columns=["(a)"] + [f"x0 = {col}" for col in kolumny])

,(a),x0 = 0,x0 = 0.2,x0 = 0.4,x0 = 0.6,x0 = 0.8,x0 = 1
0,RD,0.25000,0.62042,1.1837,2.0480,3.3775,5.4208
1,RA,0.25000,0.62042,1.1833,2.0431,3.3480,5.3000
2,RN,0.25,0.55000,0.97428,1.58236,2.45972,3.72872


### Podpunkt **(b)**


In [167]:
f = (x - 1) + (x + 1) * y(x)
y0 = 0
x0 = 0

In [168]:
kolumny = [0, 0.2, 0.4, 0.6, 0.8, 1]
wiersze = ["RD", "RA", "RN"]
wartości = [
    [fun_RD(f, x0, y0).subs(x, x_val).evalf(5).rhs for x_val in kolumny],
    [fun_RA(f, x0, y0).subs(x, x_val).evalf(5) for x_val in kolumny],
    [round(fun_RN(f, x0, y0, x_val), 5) for x_val in kolumny],
]
table_data = [(wiersze[i], *wartości[i]) for i in range(len(wiersze))]
pd.DataFrame(table_data, columns=["(b)"] + [f"x0 = {col}" for col in kolumny])

,(b),x0 = 0,x0 = 0.2,x0 = 0.4,x0 = 0.6,x0 = 0.8,x0 = 1
0,RD,0,-0.20282,-0.42446,-0.69114,-1.0441,-1.5526
1,RA,0,-0.20283,-0.42432,-0.68928,-1.0321,-1.5000
2,RN,0,-0.20000,-0.40800,-0.64224,-0.92776,-1.30175
